使用sobel算子，源码的形式实现边缘检测

1. 搞清楚，sobel算子的算法原理

涉及的几个概念: 1. 算子模板 2. 图片卷积 3. 阈值判决

- sobel的算子模板有水平和竖直方向上两种。
```
[1  2  1         [  1 0 -1
 0  0  0           2 0 -2
-1 -2 -1 ]          1 0 -1 ]
```
(3,3)的矩阵,第一个矩阵第一行(1,2,1) 第三行(-1,-2,-1)

第二个矩阵竖直方向第一列(1,2,1)，第三列为(-1,-2,-1)

- 图片的卷积 和 矩阵运算不是一回事

矩阵是行列式相乘，卷积是这个算子方框中的每一个元素对应相乘之后再求和。

举个例子:

原来的图片 `[1 2 3 4]` 每个点是一个像素

当前的计算模板是 `[a b c d]`

卷积之后的结果(对应元素相乘):

```
a*1+b*2+c*3+d*4 = dst
```

这个运算得到的结果，就是我们所说的梯度。梯度有水平和竖直两种情况。

使用arctan计算梯度的方向

使用`sqrt(a*a+b*b)` 计算梯度的浮值 = (f>th)?边缘:非边缘

>这里的a，我们的第一个算子称为竖直方向上的算子，竖直方向的算子与图片卷积得到的dst 就是a

- b就是水平方向上的算子与图片卷积得到的dst

In [1]:
import cv2
import numpy as np
import math
img = cv2.imread('image0.jpg',1)
imgInfo = img.shape
height = imgInfo[0]
width = imgInfo[1]
cv2.imshow('src',img)
        
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# 最终的边缘检测图片
dst = np.zeros((height,width,1),np.uint8)

# 遍历图片中的每一个点
# 我们的算子模板在最外围时会溢出
for i in range(0,height-2):
    for j in range(0,width-2):
        # 计算水平方向和竖直方向的梯度
        # 竖直方向的梯度
        # 取原图片 gray[i,j] gray[i,j+1] gray[i,j+2] gray[i+2,j] gray[i+2,j+1] gray[i+2,j+2] 六个点。乘以模板中对应元素
        # 其实是取九个点，因为第二行为0.不用写出来
        gy = gray[i,j]*1+gray[i,j+1]*2+gray[i,j+2]*1-gray[i+2,j]*1-gray[i+2,j+1]*2-gray[i+2,j+2]*1
        gx = gray[i,j]+gray[i+1,j]*2+gray[i+2,j]-gray[i,j+2]-gray[i+1,j+2]*2-gray[i+2,j+2]
        # 计算梯度
        grad = math.sqrt(gx*gx+gy*gy)
        # 大于阈值
        if grad>50:
            dst[i,j] = 255
        else:
            dst[i,j] = 0
cv2.imshow('dst',dst)
cv2.waitKey(0)

255